<a href="https://colab.research.google.com/github/yoursemicolon/hydromon-machine-learning/blob/main/Testing_with_tflite_Interpreter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Predict control action in hydroponic system**

# **Import Library**

In [4]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

print(tf.__version__)

2.8.2


## **Define Recommendation & Action**

In [59]:
# action yang akan dilakukan oleh actuator
action = {
    '0' : 'No Action',
    '1' : 'Pump Water',
    '2' : 'Light On',
    '3' : 'Pump Nutrient',
    '4' : 'pH Up',
    '5' : 'pH Down'
}

# recommendation berupa action yg dapat dilakukan
labels = {
    '0' : {
        'recommendation' : [0]
    },
    '1' : {
        'recommendation' : [1]
    },
    '2' : {
        'recommendation' : [2]
    },
    '3' : {
        'recommendation' : [3]
    },
    '4' : {
        'recommendation' : [4]
    },
    '5' : {
        'recommendation' : [5]
    },
    '6' : {
        'recommendation' : [1, 2]
    },
    '7' : {
        'recommendation' : [1, 4]
    },
    '8' : {
        'recommendation' : [1, 5]
    },
    '9' : {
        'recommendation' : [1, 2, 4]
    },
    '10' : {
        'recommendation' : [1, 2, 5]
    },
    '11' : {
        'recommendation' : [2, 3]
    },
    '12' : {
        'recommendation' : [2, 4]
    },
    '13' : {
        'recommendation' : [2, 5]
    },
    '14' : {
        'recommendation' : [2, 3, 4]
    },
    '15' : {
        'recommendation' : [2, 3, 5]
    },
    '16' : {
        'recommendation' : [3, 4]
    },
    '17' : {
        'recommendation' : [3, 5]
    }
}

In [8]:
# test predict
# row = [0.490833, 0.861667, 0.364933]
# predict_test = model.predict([row], verbose=0)
# print(argmax(predict_test))

## **Test the Model with TFLite Interpreter**

### **Load Model**

In [2]:
tflite_model_file = '/content/model_ver1.tflite'

In [6]:
# Load TFLite model and allocate tensors
with open(tflite_model_file, 'rb') as fid:
  tflite_model = fid.read()

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()
print("Model Loaded Succesfully.")

Model Loaded Succesfully.


In [8]:
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [9]:
input_details

[{'dtype': numpy.float32,
  'index': 0,
  'name': 'serving_default_dense_10_input:0',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
   'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32)},
  'shape': array([1, 3], dtype=int32),
  'shape_signature': array([-1,  3], dtype=int32),
  'sparsity_parameters': {}}]

In [25]:
output_details

[{'dtype': numpy.float32,
  'index': 16,
  'name': 'StatefulPartitionedCall:0',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
   'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32)},
  'shape': array([ 1, 18], dtype=int32),
  'shape_signature': array([-1, 18], dtype=int32),
  'sparsity_parameters': {}}]

In [11]:
print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

== Input details ==
name: serving_default_dense_10_input:0
shape: [1 3]
type: <class 'numpy.float32'>


In [12]:
print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])


== Output details ==
name: StatefulPartitionedCall:0
shape: [ 1 18]
type: <class 'numpy.float32'>


In [120]:
test_data = np.array([[0.490833, 0.861667, 0.364933]]) # label 12
test_data = test_data.astype('float32')
print(test_data.shape)
print(test_data.dtype)

(1, 3)
float32


In [121]:
def classify_data(interpreter, data):
  # run inference
  interpreter.set_tensor(input_details[0]['index'], data)
  interpreter.invoke()

  output_details = interpreter.get_output_details()[0]['index']
  output = interpreter.get_tensor(output_details)

  return output

In [122]:
import time
from numpy import argmax 

# classify the parameters
time_start = time.time()
prediction = classify_data(interpreter, test_data)
time_finish = time.time()
classification_time = np.round(time_finish-time_start, 5)
print(f'Classification time = {classification_time} seconds')

# print("Predicted label is : ", classification_label, ", with accuracy : ", np.round(prob*100, 2), "%.")
print("Predicted label is : ", argmax(prediction))

Classification time = 0.00034 seconds
Predicted label is :  12


## **Load Data for Other Testing**

In [113]:
import pandas as pd
import requests
from bs4 import BeautifulSoup, SoupStrainer

html = requests.get('https://github.com/HydroMon/hydromon-machine-learning/tree/main/dataset/ver2_training')

dfs = []
for link in BeautifulSoup(html.text, parse_only=SoupStrainer('a')):
    if hasattr(link, 'href') and link['href'].endswith('.csv'):
        url = 'https://github.com'+ link['href'].replace('/blob/', '/raw/')
        dfs.append(pd.read_csv(url, nrows=100))
df = pd.concat(dfs)

In [72]:
# use certain features only
# df = df.loc[:, ["TDS", "Light Intensity", "pH", "Label"]]
# df

In [114]:
# data shuffling
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,Date,Time,ID,TDS,Light Intensity,pH,Air Temperature,Humidity,EC,Label
0,2022-05-20,17:10:55,40TWL4,700,424,5.041,24,95,2.446,4
1,2022-05-20,19:59:25,40TWL4,1042,827,5.935,35,63,2.182,6
2,2022-05-20,07:05:05,40TWL4,363,870,4.829,39,57,2.229,14
3,2022-05-20,18:38:35,40TWL4,706,593,6.720,26,74,3.090,5
4,2022-05-20,13:01:05,40TWL4,1390,547,5.956,31,81,2.162,1


In [115]:
test_data = np.array([[0.490833, 0.861667, 0.364933]]) # label 12
test_data = test_data.astype('float32')
print(test_data.shape)
print(test_data.dtype)

(1, 3)
float32


In [116]:
from sklearn import preprocessing 

# min max scaler
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0,1))

In [117]:
df.iloc[:20]

,Date,Time,ID,TDS,Light Intensity,pH,Air Temperature,Humidity,EC,Label
0,2022-05-20,17:10:55,40TWL4,700,424,5.041,24,95,2.446,4
1,2022-05-20,19:59:25,40TWL4,1042,827,5.935,35,63,2.182,6
2,2022-05-20,07:05:05,40TWL4,363,870,4.829,39,57,2.229,14
3,2022-05-20,18:38:35,40TWL4,706,593,6.720,26,74,3.090,5
4,2022-05-20,13:01:05,40TWL4,1390,547,5.956,31,81,2.162,1
5,2022-05-20,02:58:20,40TWL4,648,800,5.521,32,6,3.396,11
6,2022-05-20,17:11:10,40TWL4,705,445,4.317,36,72,2.261,4
7,2022-05-20,11:35:20,40TWL4,985,404,6.435,33,50,3.041,0
8,2022-05-20,15:47:25,40TWL4,436,531,5.671,35,9,3.024,3
9,2022-05-20,11:37:55,40TWL4,839,743,5.527,26,55,3.307,0


In [127]:
# classify the parameters
counter = 0
number_of_data = df.iloc[:20]
iterate = number_of_data.iterrows()

for i, data in iterate:
  row = [data['TDS'], data['Light Intensity'], data['pH']]

  # normalize input
  norm_row = min_max_scaler.fit_transform([row])
  norm_row = np.array(norm_row)
  norm_row = norm_row.astype('float32')
  # norm_row = np.array([[0.556667, 0.458333, 0.40550]]) # label 12
  # norm_row = test_data.astype('float32')

  time_start = time.time()
  prediction = classify_data(interpreter, norm_row)
  time_finish = time.time()
  classification_time = np.round(time_finish-time_start, 5)
  
  if data['Label'] == argmax(prediction):
    print("TRUE")
    counter = counter + 1
    print(f"Actual label is {str(data['Label'])}")
    print("Predicted label is : ", argmax(prediction))
    print(f'Classification time = {classification_time} seconds')
    print()  

print(f"{counter} correct prediction from {len(number_of_data)} iteration")

TRUE
Actual label is 16
Predicted label is :  16
Classification time = 5e-05 seconds

TRUE
Actual label is 16
Predicted label is :  16
Classification time = 6e-05 seconds

2 correct prediction from 20 iteration
